In [5]:
from os import environ, chdir

from keras.preprocessing.image import ImageDataGenerator

from keras import models, layers, optimizers, callbacks

import tensorflow as tf

import h5py

environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
chdir(r'D:\hackathon')

In [6]:
# Setting Image and Data Generators
train_idg = ImageDataGenerator(rescale=1. / 255,
                               zoom_range=[1.0, 1.25],
                               width_shift_range=0.1,
                               height_shift_range=0.1,
                               fill_mode='reflect')

train_g = train_idg.flow_from_directory(directory=r'train1',
                                        target_size=(28, 28),
                                        class_mode='binary',
                                        batch_size=108,
                                        shuffle=True)

valid_idg = ImageDataGenerator(rescale=1. / 255)

valid_g = valid_idg.flow_from_directory(directory=r'valid1',
                                        target_size=(28, 28),
                                        class_mode='binary',
                                        batch_size=50,
                                        shuffle=True)

Found 1080 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


In [7]:
my_model = models.Sequential()
my_model.add(layers.Conv2D(filters=64, kernel_size=(3,3), strides=(1, 1),
                           input_shape=(28, 28, 1),padding=1))
my_model.add(layers.Activation('relu'))
my_model.add(layers.MaxPooling2D(pool_size=(2, 2)))

my_model.add(layers.Conv2D(filters=128, kernel_size=(3,3), strides=(1, 1),padding=1))
my_model.add(layers.Activation('relu'))
my_model.add(layers.MaxPooling2D(pool_size=(2, 2)))



my_model.add(layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1, 1),padding=1))
my_model.add(layers.Activation('relu'))
my_model.add(layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1, 1),padding=1))
my_model.add(layers.Activation('relu'))
my_model.add(layers.MaxPooling2D(pool_size=(2, 2)))
my_model.add(layers.Flatten())
my_model.add(layers.Dense(units=1024))
print(my_model.summary())


AttributeError: 'int' object has no attribute 'lower'

In [3]:
# CNN Architecture
my_model = models.Sequential()
my_model.add(layers.Conv2D(filters=16, kernel_size=(2,2), strides=(1, 1),
                           input_shape=(100, 100, 3)))
my_model.add(layers.Activation('relu'))
my_model.add(layers.Conv2D(filters=16, kernel_size=(2, 2), strides=(1, 1)))
my_model.add(layers.Activation('relu'))
my_model.add(layers.MaxPooling2D(pool_size=(2, 2)))

my_model.add(layers.Dropout(rate=0.4))

my_model.add(layers.Conv2D(filters=16, kernel_size=(2, 2), strides=(1, 1)))
my_model.add(layers.Activation('relu'))
my_model.add(layers.Conv2D(filters=16, kernel_size=(2, 2), strides=(1, 1)))
my_model.add(layers.Activation('relu'))
my_model.add(layers.MaxPooling2D(pool_size=(2, 2)))

my_model.add(layers.Dropout(rate=0.4))

my_model.add(layers.Conv2D(filters=16, kernel_size=(2, 2), strides=(1, 1)))
my_model.add(layers.Activation('relu'))
my_model.add(layers.MaxPooling2D(pool_size=(2, 2)))
my_model.add(layers.Conv2D(filters=16, kernel_size=(2, 2), strides=(1, 1)))
my_model.add(layers.Activation('relu'))
my_model.add(layers.MaxPooling2D(pool_size=(2, 2)))

my_model.add(layers.Flatten())

my_model.add(layers.Dropout(rate=0.4))

my_model.add(layers.Dense(units=10))
my_model.add(layers.Activation('relu'))

my_model.add(layers.Dense(units=1))
my_model.add(layers.Activation('sigmoid'))
print(my_model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 99, 99, 16)        208       
_________________________________________________________________
activation_1 (Activation)    (None, 99, 99, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 98, 98, 16)        1040      
_________________________________________________________________
activation_2 (Activation)    (None, 98, 98, 16)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 49, 49, 16)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 49, 49, 16)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 48, 48, 16)        1040      
__________

In [9]:
# Model Loss function and Optimizer method
compile = my_model.compile(optimizer=optimizers.Adadelta(lr=0.15), loss='binary_crossentropy',
                           metrics=['accuracy'])


In [12]:
# Settting Callbacks
chdir(r'D:\hackathon')
check_p = callbacks.ModelCheckpoint(filepath='hydrangea_cnn_{val_acc:.2f}.h5',
                                    monitor='val_acc', verbose=1,
                                    save_best_only=True, save_weights_only=False)
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_acc', factor=0.95, patience=3,
                                        verbose=1, cooldown=2)
callb_l = [check_p, reduce_lr]

In [ ]:
# Training Options
fit = my_model.fit_generator(generator=train_g, steps_per_epoch=22, epochs=100, verbose=1,
                             callbacks=callb_l, validation_data=valid_g, validation_steps=4)

Epoch 1/100
21/22 [===========================>..] - ETA: 5s - loss: 0.6942 - acc: 0.4907 
Epoch 00001: val_acc improved from -inf to 0.50000, saving model to hydrangea_cnn_0.50.h5
22/22 [==============================] - 118s 5s/step - loss: 0.6940 - acc: 0.4937 - val_loss: 0.6931 - val_acc: 0.5000
Epoch 2/100
21/22 [===========================>..] - ETA: 4s - loss: 0.6939 - acc: 0.4824
Epoch 00002: val_acc did not improve
22/22 [==============================] - 98s 4s/step - loss: 0.6939 - acc: 0.4832 - val_loss: 0.6929 - val_acc: 0.5000
Epoch 3/100
14/22 [==================>...........] - ETA: 36s - loss: 0.6935 - acc: 0.5040

In [ ]:
# Saving Model
my_model.save(filepath=r'hydrangea_cnn.h5', overwrite=True)


In [13]:

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs= range(len(acc))

plt.plot(epochs, acc, 'b', label='training acc')
plt.plot(epochs, val_acc, 'r', label='validation loss')
plt.title('training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'b', label='training loss')
plt.plot(epochs, val_loss, 'r', label='training loss')
plt.title('training and validation loss')
plt.legend()

plt.show()



NameError: name 'history' is not defined